In [13]:
import pandas as pd
import numpy as np
import torch.nn as nn
import yfinance as yf


In [11]:
# Download Coinbase (COIN) historical price data
ticker = 'COIN'
stock = yf.download(ticker, progress=False)
display(stock.head(1))

C:\Users\saris\AppData\Local\Temp\ipykernel_27596\3169291809.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock = yf.download(ticker, progress=False)


Price            Close        High    Low   Open    Volume
Ticker            COIN        COIN   COIN   COIN      COIN
Date                                                      
2021-04-14  328.279999  429.540009  310.0  381.0  81065700

In [ ]:
def add_features(df):

    # 30 day and 200 day moving average and standard deviation
    df['MA30'] = df['Close'].rolling(window=30).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    df['STD30'] = df['Close'].rolling(window=30).std()
    df['STD200'] = df['Close'].rolling(window=200).std()

    # 14 day RSI
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI14'] = 100 - (100 / (1 + rs))

    # 14 day MACD
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']  

    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['Signal'] 
    df['MACD_Signal'] = np.where(df['MACD_Hist'] > 0, 1, -1)
    df['MACD_Signal'] = df['MACD_Signal'].shift(1)  # Shift to avoid lookahead bias
    df['MACD_Signal'] = df['MACD_Signal'].fillna(0)  # Fill NaN values with 0
    df['MACD_Signal'] = df['MACD_Signal'].astype(int)  # Ensure it's an integer type

    # 14 day Stochastic Oscillator
    df['L14'] = df['Low'].rolling(window=14).min()  
    df['H14'] = df['High'].rolling(window=14).max()
    df['Stoch'] = (df['Close'] - df['L14']) / (df['H14'] - df['L14']) * 100
    df['Stoch_Signal'] = np.where(df['Stoch'] > 80, -1, np.where(df['Stoch'] < 20, 1, 0))
    df['Stoch_Signal'] = df['Stoch_Signal'].shift(1)  # Shift to avoid lookahead bias
    df['Stoch_Signal'] = df['Stoch_Signal'].fillna(0)  # Fill NaN values with 0
    df['Stoch_Signal'] = df['Stoch_Signal'].astype(int)

    # daily and weekly demark indicators
    df['Demark_Daily'] = np.where(df['Close'] > df['Open'], 1, -1)
    df['Demark_Weekly'] = df['Demark_Daily'].rolling(window=7).sum()
    df['Demark_Weekly'] = np.where(df['Demark_Weekly'] > 0, 1, -1)
    df['Demark_Weekly'] = df['Demark_Weekly'].shift(1)  # Shift to avoid lookahead bias
    df['Demark_Weekly'] = df['Demark_Weekly'].fillna(0)  # Fill NaN values with 0
    df['Demark_Weekly'] = df['Demark_Weekly'].astype(int)

    # daily and weekly Williams %R
    df['Williams_Daily'] = (df['H14'] - df['Close'])
    df['Williams_Daily'] /= (df['H14'] - df['L14'])
    df['Williams_Daily'] = df['Williams_Daily'] * -100
    df['Williams_Weekly'] = df['Williams_Daily'].rolling(window=7).mean()
    df['Williams_Weekly'] = np.where(df['Williams_Weekly'] > -20, 1,                                  np.where(df['Williams_Weekly'] < -80, -1, 0))
    df['Williams_Weekly'] = df['Williams_Weekly'].shift(1)  # Shift to avoid lookahead bias
    df['Williams_Weekly'] = df['Williams_Weekly'].fillna(0)
    df['Williams_Weekly'] = df['Williams_Weekly'].astype(int)


    return df






NameError: name 'stockcoinbase' is not defined

In [ ]:
# Visualize the distribution of option prices
plt.figure(figsize=(10, 6))
sns.histplot(options_data['option_price'], bins=30, kde=True)
plt.title('Distribution of Option Prices')
plt.xlabel('Option Price')
plt.ylabel('Frequency')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 8))
correlation_matrix = options_data.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

## Insights

1. The distribution of option prices shows...
2. The correlation heatmap indicates...

Further analysis can be conducted to explore relationships between different features and their impact on option pricing.